# Higher-Order Singular Value Decomposition (HOSVD)

This notebook performs HOSVD on hydrogen jet combustion simulation data.

## Import Libraries

In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import tensorly as tl
from tensorly.decomposition import tucker
from tensorly.tucker_tensor import tucker_to_tensor
from tensorly.tenalg import mode_dot, multi_mode_dot
import kagglehub
import h5py
from tqdm import tqdm
import pandas as pd


## Configuration Parameters

In [2]:
n_snapshots = 200
subsample_x = 10
subsample_y = 10
paths = ["sharmapushan/hydrogen-jet-5000","sharmapushan/hydrogen-jet-10000", "sharmapushan/nonreacting-hydrogen-jet-5000","sharmapushan/nonreacting-hydrogen-jet-10000"]
data_paths = [kagglehub.dataset_download(name) for name in paths]
data_path = data_paths[0]
print(f"Dataset path: {data_path}")


Dataset path: /home/isacco/.cache/kagglehub/datasets/sharmapushan/hydrogen-jet-5000/versions/7


## Load Metadata and Grid

In [3]:
# Load metadata
with open(data_path + '/info.json') as f:
    metadata = json.load(f)

Nx, Ny = metadata['global']['Nxyz']

# Load grid
X_filename = metadata['global']['grid']['x']
Y_filename = metadata['global']['grid']['y']
X = np.fromfile(data_path + '/' + X_filename, dtype='<f4').reshape(Ny, Nx)
Y = np.fromfile(data_path + '/' + Y_filename, dtype='<f4').reshape(Ny, Nx)

# Subsample grid
X_sub = X[::subsample_x, ::subsample_y]
Y_sub = Y[::subsample_x, ::subsample_y]
Nx_sub, Ny_sub = X_sub.shape

print(f"Original grid: ({Nx}, {Ny})")
print(f"Subsampled grid: ({Nx_sub}, {Ny_sub})")

Original grid: (1600, 2000)
Subsampled grid: (200, 160)


## Define Components and Initialize Tensor

In [4]:
# Define components to load
component_names = ['YH', 'YH2', 'YO', 'YO2', 'YOH', 'YH2O', 'YHO2', 'YH2O2']
n_components = len(component_names)

# Molar masses (g/mol)
molar_masses = {
    'YH': 1.0,
    'YH2': 2.0,
    'YO': 8.0,
    'YO2': 16.0,
    'YOH': 9.0,
    'YH2O': 10.0,
    'YHO2': 17.0,
    'YH2O2': 18.0
}


# Mapping from component names to file keys
file_key_map = {
    #'RHO': 'RHO_kgm-3 filename',
    #'UX': 'UX_ms-1 filename',
    #'UY': 'UY_ms-1 filename',
    #'T': 'T_K filename',
    'YH': 'YH filename',
    'YH2': 'YH2 filename',
    'YO': 'YO filename',
    'YO2': 'YO2 filename',
    'YOH': 'YOH filename',
    'YH2O': 'YH2O filename',
    'YHO2': 'YHO2 filename',
    'YH2O2': 'YH2O2 filename'
}

## Load Snapshot Data

In [5]:
# Load Snapshot Data
def load_dataset(data_path, component_names, file_key_map, Ny, Nx, subsample_x, subsample_y):
    with open(f"{data_path}" + '/info.json', 'r') as f:
        metadata = json.load(f)

    # First pass: identify which components are available in this dataset
    available_components = []
    available_indices = []
    for c_idx, comp_name in enumerate(component_names):
        filename_key = file_key_map[comp_name]
        if filename_key in metadata['local'][0]:
            available_components.append(comp_name)
            available_indices.append(c_idx)

    print(f"  Available components in dataset: {available_components}")
    n_available = len(available_components)

    # Initialize tensor with only available components: (x, y, components, time)
    tensor = np.zeros((Nx_sub, Ny_sub, n_available, n_snapshots))
    for t_idx in range(n_snapshots):
        for new_idx, (comp_name, orig_idx) in enumerate(zip(available_components, available_indices)):
            filename_key = file_key_map[comp_name]
            filename = metadata['local'][t_idx][filename_key]
            data = np.fromfile(f"{data_path}/{filename}", dtype='<f4').reshape(Ny, Nx)
            # Convert mass fraction to molar fraction by dividing by molar mass
            molar_data = data / molar_masses[comp_name]
            tensor[:, :, new_idx, t_idx] = molar_data[::subsample_x, ::subsample_y]

    return tensor
tensors = {path: load_dataset(path, component_names, file_key_map, Ny, Nx, subsample_x, subsample_y)
           for path in data_paths}

  Available components in dataset: ['YH', 'YH2', 'YO', 'YO2', 'YOH', 'YH2O', 'YHO2', 'YH2O2']
  Available components in dataset: ['YH', 'YH2', 'YO', 'YO2', 'YOH', 'YH2O', 'YHO2', 'YH2O2']
  Available components in dataset: ['YH2', 'YO2']
  Available components in dataset: ['YH2', 'YO2']


In [6]:
for dataset_path, tensor in tensors.items():
    if "nonreact" not in dataset_path:
        print("\n" + "=" * 80)
        print(f"Dataset: {dataset_path}")
        print("=" * 80)
        print(f"{'Component':<15} {'Mean':<12} {'Std':<12} {'Min':<12} {'Max':<12}")
        print("-" * 80)
        
        # Compute per-component stats
        for c_idx, comp_name in enumerate(component_names):
            component_data = tensor[:, :, c_idx, :]
            mean_val = np.mean(component_data)
            std_val = np.std(component_data)
            min_val = np.min(component_data)
            max_val = np.max(component_data)
            
            print(f"{comp_name:<15} {mean_val:<12.6e} {std_val:<12.6e} {min_val:<12.6e} {max_val:<12.6e}")
        
        print("-" * 80)
        
        # Overall tensor stats
        print(f"Overall Tensor Statistics:")
        print(f"  Shape: {tensor.shape}")
        print(f"  Total elements: {tensor.size:,}")
        print(f"  Memory size: {tensor.nbytes / (1024**2):.2f} MB")
        print(f"  Global mean: {np.mean(tensor):.6e}")
        print(f"  Global std:  {np.std(tensor):.6e}")
        print(f"  Global min:  {np.min(tensor):.6e}")
        print(f"  Global max:  {np.max(tensor):.6e}")
        print("=" * 80)



Dataset: /home/isacco/.cache/kagglehub/datasets/sharmapushan/hydrogen-jet-5000/versions/7
Component       Mean         Std          Min          Max         
--------------------------------------------------------------------------------
YH              2.079209e-04 4.148193e-04 -3.964218e-15 3.615513e-03
YH2             2.953624e-02 7.822950e-02 0.000000e+00 3.249700e-01
YO              8.354195e-05 2.403193e-04 -6.604444e-17 1.807642e-03
YO2             1.065733e-02 5.989962e-03 2.523242e-07 1.456250e-02
YOH             1.958776e-04 5.469642e-04 -1.057820e-17 2.615134e-03
YH2O            4.196671e-03 7.683622e-03 -1.294728e-15 2.352162e-02
YHO2            4.815844e-07 1.290229e-06 -5.399450e-15 1.326961e-05
YH2O2           2.095132e-08 4.894754e-08 -8.448455e-20 7.321171e-07
--------------------------------------------------------------------------------
Overall Tensor Statistics:
  Shape: (200, 160, 8, 200)
  Total elements: 51,200,000
  Memory size: 390.62 MB
  Global mean: 5.609

## Variable Statistics

In [7]:
component_means_all = {}  # dict to store mean values per dataset

for dataset_path, tensor in tensors.items():
    if "nonreact" not in dataset_path:
        print("\n" + "=" * 100)
        print(f"Dataset: {dataset_path}")
        print("Rescaling chemical species by their mean values (excluding zeros)...")
        print("=" * 100)
        print(f"{'Component':<15} {'Original Mean':<15} {'Non-zero %':<12} {'Scaled Mean':<15} {'Scaled Max':<15}")
        print("-" * 100)
        
        print("\n=== Outlier Detection and Scaling ===")
        max_values = np.zeros(n_components)
        
        for c_idx, comp_name in enumerate(component_names):
            
            component_data = tensor[:, :, c_idx, :]

            # Outlier detection using IQR method
            q1 = np.percentile(component_data, 25)
            q3 = np.percentile(component_data, 75)
            iqr = q3 - q1
            lower_bound = q1 - 3 * iqr
            upper_bound = q3 + 3 * iqr

            outliers = (component_data < lower_bound) | (component_data > upper_bound)
            n_outliers = np.sum(outliers)
            outlier_percentage = 100 * n_outliers / component_data.size

            print(f"{comp_name}:")
            print(f"  Range: [{component_data.min():.2e}, {component_data.max():.2e}]")
            print(f"  Q1={q1:.2e}, Q3={q3:.2e}, IQR={iqr:.2e}")
            print(f"  Outliers: {n_outliers} ({outlier_percentage:.2f}%) beyond [{lower_bound:.2e}, {upper_bound:.2e}]")
            # Scale by maximum value
            max_values[c_idx] = component_data.max()
            if max_values[c_idx] > 0:
                tensor[:, :, c_idx, :] /= max_values[c_idx]
                print(f"  Scaled by max value: {max_values[c_idx]:.2e}")
            else:
                print(f"  WARNING: Max value is zero, skipping scaling")


print("Scaling complete.\n")

    




Dataset: /home/isacco/.cache/kagglehub/datasets/sharmapushan/hydrogen-jet-5000/versions/7
Rescaling chemical species by their mean values (excluding zeros)...
Component       Original Mean   Non-zero %   Scaled Mean     Scaled Max     
----------------------------------------------------------------------------------------------------

=== Outlier Detection and Scaling ===
YH:
  Range: [-3.96e-15, 3.62e-03]
  Q1=4.01e-23, Q3=1.72e-04, IQR=1.72e-04
  Outliers: 962161 (15.03%) beyond [-5.15e-04, 6.87e-04]
  Scaled by max value: 3.62e-03
YH2:
  Range: [0.00e+00, 3.25e-01]
  Q1=1.55e-14, Q3=1.39e-03, IQR=1.39e-03
  Outliers: 1382255 (21.60%) beyond [-4.17e-03, 5.56e-03]
  Scaled by max value: 3.25e-01
YO:
  Range: [-6.60e-17, 1.81e-03]
  Q1=4.09e-18, Q3=3.85e-06, IQR=3.85e-06
  Outliers: 1268679 (19.82%) beyond [-1.16e-05, 1.54e-05]
  Scaled by max value: 1.81e-03
YO2:
  Range: [2.52e-07, 1.46e-02]
  Q1=3.88e-03, Q3=1.46e-02, IQR=1.07e-02
  Outliers: 0 (0.00%) beyond [-2.82e-02, 4.66e-02]

## Perform HOSVD Decomposition

In [9]:
decomposition_results = {}  # store factors and cores per dataset

for dataset_path, tensor in tensors.items():
    print("\n" + "=" * 100)
    print(f"Performing HOSVD for dataset: {dataset_path}")
    print("=" * 100)
    
    factors = []
    for i in tqdm(range(tensor.ndim)):
        U, _, _ = np.linalg.svd(tl.unfold(tensor, mode=i), full_matrices=False)
        factors.append(U)

    core = multi_mode_dot(tensor, [U.T for U in factors], modes=range(tensor.ndim))
    print("done")
    reconst = multi_mode_dot(core, factors, modes=range(tensor.ndim))
    
    # check reconstruction accuracy
    close = np.allclose(reconst, tensor, rtol=1e-5, atol=1e-8)
    error = np.linalg.norm(np.subtract(reconst, tensor)) / np.linalg.norm(tensor)
    
    print(f"  Reconstruction close: {close}")
    print(f"  Relative reconstruction error: {error:.6e}")
    print("-" * 100)
    
    # store results
    decomposition_results[dataset_path] = {
        "core": core,
        "factors": factors,
        #"reconstruction_error": error
    }
    # Save
    with h5py.File(f"./HOSVD_DECOMPOSITIONS/HOSVD_decomposition_{dataset_path.split('/')[-3]}.h5", 'w') as f:
        f.create_dataset('core', data=core)
        for i, factor in enumerate(factors):
            f.create_dataset(f'factor_{i}', data=factor)


print("\n" + "=" * 100)
print("=" * 100)



Performing HOSVD for dataset: /home/isacco/.cache/kagglehub/datasets/sharmapushan/hydrogen-jet-5000/versions/7


100%|██████████| 4/4 [00:16<00:00,  4.04s/it]


done
  Reconstruction close: True
  Relative reconstruction error: 2.686779e-15
----------------------------------------------------------------------------------------------------

Performing HOSVD for dataset: /home/isacco/.cache/kagglehub/datasets/sharmapushan/hydrogen-jet-10000/versions/2


100%|██████████| 4/4 [00:16<00:00,  4.04s/it]


done
  Reconstruction close: True
  Relative reconstruction error: 1.807233e-15
----------------------------------------------------------------------------------------------------

Performing HOSVD for dataset: /home/isacco/.cache/kagglehub/datasets/sharmapushan/nonreacting-hydrogen-jet-5000/versions/3


100%|██████████| 4/4 [00:02<00:00,  1.72it/s]


done
  Reconstruction close: True
  Relative reconstruction error: 1.761022e-15
----------------------------------------------------------------------------------------------------

Performing HOSVD for dataset: /home/isacco/.cache/kagglehub/datasets/sharmapushan/nonreacting-hydrogen-jet-10000/versions/2


100%|██████████| 4/4 [00:02<00:00,  1.56it/s]


done
  Reconstruction close: True
  Relative reconstruction error: 2.118009e-15
----------------------------------------------------------------------------------------------------

